In [5]:
import pandas as pd
import numpy as np

import sklearn
from sklearn import tree

import torch
from torch import nn
from torch.utils.data import DataLoader


import matplotlib.pyplot as plt

%matplotlib inline

pd.options.display.float_format = '{:,.2f}'.format

In [6]:
df = pd.read_csv('../data/processed/all_rooms_combined.csv', index_col=0)

print(f'Наблюдений в датасете: {df.shape[0]}')

Наблюдений в датасете: 16208


In [7]:
df.head()

,rooms,subway,admin_okrug,district,street,home_number,price,year_of_construction,total_meters,kitchen_meters,flat_type,house_type,"dist_to_subway, min",way_to_subway,is_euro,is_skyscraper,floor_type,wc_count,wc_type,class_real
0,1,Спартак,СЗАО,р-н Покровское-Стрешнево,Алиа ЖК,к7,"16,781,328.00","2,023.00",37.00,10.00,Новостройка,Монолитный,5.00,пешком,False,False,usual,1,совмещенный,премиум
1,1,Шелепиха,СЗАО,р-н Хорошево-Мневники,Шелепихинская набережная,34к7,"16,500,000.00","2,024.00",42.00,12.00,Новостройка,Монолитный,14.00,пешком,False,False,usual,1,совмещенный,бизнес
2,1,Стрешнево,СЗАО,р-н Щукино,Щукинская улица,7/9С7,"20,540,352.00","2,023.00",47.00,16.00,Новостройка,Монолитный,10.00,пешком,False,False,usual,2,совмещенный,премиум
3,1,Шелепиха,СЗАО,р-н Хорошево-Мневники,Шелепихинская набережная,34к3,"20,800,000.00","2,020.00",32.00,10.00,Вторичка,Монолитный,14.00,пешком,False,False,view,1,совмещенный,премиум
4,1,Спартак,СЗАО,р-н Покровское-Стрешнево,Северо-Западный ао,Клубный Город на Реке Примавера ЖК,"30,460,120.00","2,024.00",52.00,11.00,Новостройка,Монолитный,14.00,пешком,False,False,usual,1,раздельный,премиум


In [8]:
df.dtypes

rooms                     int64
subway                   object
admin_okrug              object
district                 object
street                   object
home_number              object
price                   float64
year_of_construction    float64
total_meters            float64
kitchen_meters          float64
flat_type                object
house_type               object
dist_to_subway, min     float64
way_to_subway            object
is_euro                    bool
is_skyscraper              bool
floor_type               object
wc_count                  int64
wc_type                  object
class_real               object
dtype: object

### Разбиваем на X и y

In [ ]:
log = ['total_meters', 'kitchen_meters', 'dist_to_subway, min']
categorical = ['admin_okrug', 'district', 'subway', 'is_skyscraper', 'class_real', 'way_to_subway', 'wc_type', 'house_type', 'flat_type']
ordinal = ['rooms', 'year_of_construction']

In [107]:
X = df[log + categorical + ordinal]
y = df['price']/df['total_meters']

In [108]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

In [109]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, shuffle=True)

In [110]:
X_train.head(2)

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,rooms,year_of_construction
15780,21.00,5.00,7.00,СВАО,Отрадное,False,бизнес,на транспорте,совмещенный,Панельный,Новостройка,9,"2,025.00"
1177,33.00,15.00,7.00,НАО (Новомосковский),Филатов Луг,False,комфорт,пешком,совмещенный,Монолитный,Новостройка,1,"2,025.00"


In [111]:
X_train.isna().sum()

total_meters               0
kitchen_meters             0
dist_to_subway, min        0
admin_okrug                0
subway                     0
is_skyscraper              0
class_real                 0
way_to_subway              0
wc_type                    0
house_type              1494
flat_type                  0
rooms                      0
year_of_construction     510
dtype: int64

In [112]:
X_train.house_type = X_train.house_type.fillna('No')
X_train.year_of_construction = X_train.year_of_construction.fillna(-1)
X_train['dist_to_subway, min'] = X_train['dist_to_subway, min'].fillna(20)
X_train.way_to_subway = X_train.way_to_subway.fillna('пешком')

In [113]:
X_test.house_type = X_test.house_type.fillna('No')
X_test.year_of_construction = X_test.year_of_construction.fillna(-1)
X_test['dist_to_subway, min'] = X_test['dist_to_subway, min'].fillna(20)
X_test.way_to_subway = X_test.way_to_subway.fillna('пешком')

In [114]:
X_test.isna().sum()

total_meters            0
kitchen_meters          0
dist_to_subway, min     0
admin_okrug             0
subway                  0
is_skyscraper           0
class_real              0
way_to_subway           0
wc_type                 0
house_type              0
flat_type               0
rooms                   0
year_of_construction    0
dtype: int64

In [115]:
def log_transform(x):
    return np.log(x + 1)

In [116]:
log_transformer = FunctionTransformer(log_transform)

In [117]:
col_transformer = ColumnTransformer([("Log transform", log_transformer, log),
                                ("Scale", StandardScaler(), ordinal),
                                ("One hot", OneHotEncoder(sparse=False, handle_unknown='ignore'),categorical)],
                                remainder="passthrough")
X_train_transformed = col_transformer.fit_transform(X_train)

In [118]:
X_test_transformed = col_transformer.transform(X_test)

In [119]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import catboost as ctb

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [120]:
df_test = pd.merge(X_test, pd.DataFrame(y_test, columns=['real']), how = 'left', left_index=True, right_index=True)

### Простая линейная регрессия ###

In [121]:
linreg = LinearRegression().fit(X_train_transformed, y_train)
MAE_lr = round(mean_absolute_error(y_test, linreg.predict(X_test_transformed)), 3)
MAE_lr

250768661533.34

In [122]:
df_test['preds'] = linreg.predict(X_test_transformed)
df_test.head()

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,rooms,year_of_construction,real,preds
16769,7.00,2.00,13.00,ЗАО,Солнцево,False,бизнес,пешком,совмещенный,Блочный,Вторичка Апартаменты,9,"1,978.00","321,428.57","291,328.00"
2532,37.00,10.00,7.00,ЗАО,Раменки,False,бизнес,пешком,совмещенный,Блочный,Вторичка,1,"1,980.00","327,027.03","369,344.00"
11934,169.00,25.00,9.00,ЦАО,Выставочная,True,элитный,пешком,совмещенный,Монолитный,Новостройка,5,"2,023.00","1,202,130.18","1,035,888.00"
7643,42.00,10.00,1.00,ЦАО,Электрозаводская,False,премиум,пешком,совмещенный,Монолитный,Новостройка,2,"2,024.00","442,837.14","469,344.00"
17832,21.00,5.00,15.00,ЮАО,ЗИЛ,False,премиум,пешком,совмещенный,Монолитный,Новостройка,9,"2,023.00","677,945.52","487,328.00"


In [123]:
df_test['diff'] = df_test['real']-df_test['preds']

In [124]:
df_test.sort_values(by='diff')

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,rooms,year_of_construction,real,preds,diff
9886,89.00,12.00,6.00,ЦАО,Добрынинская,False,премиум,пешком,раздельный,Кирпичный,Вторичка,3,"1,940.00","421,348.31","135,498,639,299,296.00","-135,498,638,877,947.69"
17197,18.00,3.00,16.00,СВАО,Лось,False,комфорт,на транспорте,совмещенный,Монолитный,Вторичка Апартаменты,9,"2,019.00","272,222.22","135,498,639,079,248.00","-135,498,638,807,025.78"
9662,50.00,7.00,5.00,СВАО,Лось,False,эконом,на транспорте,раздельный,Панельный,Вторичка,3,"1,969.00","230,000.00","135,498,639,020,992.00","-135,498,638,790,992.00"
1910,43.00,10.00,16.00,ЮАО,Бирюлёво-Пассажирская,False,комфорт,пешком,раздельный,Монолитный,Вторичка,1,"2,013.00","259,302.33","135,498,639,015,168.00","-135,498,638,755,865.67"
3048,39.00,9.00,6.00,ЮЗАО,Битца,False,комфорт,на транспорте,раздельный,Монолитный,Вторичка,1,"1,991.00","266,666.67","135,498,639,021,120.00","-135,498,638,754,453.33"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4571,234.00,60.00,7.00,ЦАО,Киевская,False,элитный,на транспорте,раздельный,Монолитный,Новостройка,2,"2,026.00","2,390,384.62","1,485,376.00","905,008.62"
9604,202.00,12.00,6.00,ЦАО,Парк Культуры,False,элитный,пешком,совмещенный,Кирпичный,Вторичка,3,"2,009.00","2,289,287.87","1,197,104.00","1,092,183.87"
10444,120.00,12.00,1.00,ЦАО,Лубянка,False,элитный,пешком,совмещенный,No,Вторичка Апартаменты,3,-1.00,"2,337,411.54","1,063,616.00","1,273,795.54"
13558,302.00,10.00,6.00,ЦАО,Пушкинская,False,элитный,пешком,совмещенный,No,Вторичка,5,"2,006.00","2,423,663.58","1,129,664.00","1,293,999.58"


### Ridge-регрессия ###

In [125]:
ridge = Ridge(alpha=1000).fit(X_train_transformed, y_train)
MAE_r = round(mean_absolute_error(y_test, ridge.predict(X_test_transformed)), 3)
MAE_r

74551.982

In [126]:
df_test['preds'] = ridge.predict(X_test_transformed)
df_test.head()

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,rooms,year_of_construction,real,preds,diff
16769,7.00,2.00,13.00,ЗАО,Солнцево,False,бизнес,пешком,совмещенный,Блочный,Вторичка Апартаменты,9,"1,978.00","321,428.57","286,734.19","30,100.57"
2532,37.00,10.00,7.00,ЗАО,Раменки,False,бизнес,пешком,совмещенный,Блочный,Вторичка,1,"1,980.00","327,027.03","357,154.77","-42,316.97"
11934,169.00,25.00,9.00,ЦАО,Выставочная,True,элитный,пешком,совмещенный,Монолитный,Новостройка,5,"2,023.00","1,202,130.18","934,617.36","166,242.18"
7643,42.00,10.00,1.00,ЦАО,Электрозаводская,False,премиум,пешком,совмещенный,Монолитный,Новостройка,2,"2,024.00","442,837.14","617,498.37","-26,506.86"
17832,21.00,5.00,15.00,ЮАО,ЗИЛ,False,премиум,пешком,совмещенный,Монолитный,Новостройка,9,"2,023.00","677,945.52","421,235.60","190,617.52"


In [127]:
df_test['diff'] = df_test['real']-df_test['preds']
df_test.sort_values(by='diff')

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,rooms,year_of_construction,real,preds,diff
14116,388.00,30.00,9.00,ЦАО,Шаболовская,False,комфорт,пешком,совмещенный,Монолитный,Вторичка Пентхаус,5,"2,006.00","283,247.42","622,619.34","-339,371.92"
12084,225.00,33.00,7.00,ЦАО,Октябрьская,False,бизнес,пешком,совмещенный,Кирпичный,Вторичка,5,"1,900.00","311,111.11","590,363.45","-279,252.34"
14910,183.00,18.00,1.00,ЦАО,Бауманская,False,комфорт,пешком,совмещенный,No,Вторичка,5,"1,902.00","284,153.01","561,300.57","-277,147.57"
14385,260.00,20.00,10.00,ЦАО,Шелепиха,False,комфорт,пешком,раздельный,Монолитный,Вторичка,5,"2,003.00","288,461.54","553,871.62","-265,410.08"
13581,190.00,20.00,6.00,ЦАО,Чистые пруды,False,премиум,пешком,совмещенный,No,Вторичка,5,"1,917.00","421,052.63","684,764.75","-263,712.11"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10444,120.00,12.00,1.00,ЦАО,Лубянка,False,элитный,пешком,совмещенный,No,Вторичка Апартаменты,3,-1.00,"2,337,411.54","906,592.50","1,430,819.04"
10461,82.00,6.00,13.00,ЦАО,Киевская,False,элитный,пешком,совмещенный,Монолитный,Новостройка,3,"2,024.00","2,344,408.59","904,838.72","1,439,569.86"
4571,234.00,60.00,7.00,ЦАО,Киевская,False,элитный,на транспорте,раздельный,Монолитный,Новостройка,2,"2,026.00","2,390,384.62","934,323.34","1,456,061.27"
13558,302.00,10.00,6.00,ЦАО,Пушкинская,False,элитный,пешком,совмещенный,No,Вторичка,5,"2,006.00","2,423,663.58","931,702.38","1,491,961.20"


### Lasso регрессия ###

In [128]:
lasso_reg = Lasso(alpha = 2.65e-05).fit(X_train_transformed, y_train)
MAE_lasso = round(mean_absolute_error(y_test, lasso_reg.predict(X_test_transformed)), 3)
MAE_lasso

/Users/max/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.546e+13, tolerance: 9.618e+10
  model = cd_fast.enet_coordinate_descent(


51915.973

In [129]:
df_test['preds'] = lasso_reg.predict(X_test_transformed)
df_test.head()

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,rooms,year_of_construction,real,preds,diff
16769,7.00,2.00,13.00,ЗАО,Солнцево,False,бизнес,пешком,совмещенный,Блочный,Вторичка Апартаменты,9,"1,978.00","321,428.57","291,278.97","34,694.38"
2532,37.00,10.00,7.00,ЗАО,Раменки,False,бизнес,пешком,совмещенный,Блочный,Вторичка,1,"1,980.00","327,027.03","369,383.17","-30,127.74"
11934,169.00,25.00,9.00,ЦАО,Выставочная,True,элитный,пешком,совмещенный,Монолитный,Новостройка,5,"2,023.00","1,202,130.18","1,035,907.35","267,512.82"
7643,42.00,10.00,1.00,ЦАО,Электрозаводская,False,премиум,пешком,совмещенный,Монолитный,Новостройка,2,"2,024.00","442,837.14","469,250.62","-174,661.23"
17832,21.00,5.00,15.00,ЮАО,ЗИЛ,False,премиум,пешком,совмещенный,Монолитный,Новостройка,9,"2,023.00","677,945.52","487,316.89","256,709.92"


In [130]:
df_test['diff'] = df_test['real']-df_test['preds']
df_test.sort_values(by='diff')

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,rooms,year_of_construction,real,preds,diff
12215,210.00,20.00,7.00,ЦАО,Третьяковская,False,элитный,пешком,совмещенный,Монолитный,Вторичка,5,"2,000.00","952,380.95","1,723,466.90","-771,085.95"
14907,103.00,13.00,10.00,ЦАО,Киевская,False,элитный,пешком,раздельный,Кирпичный,Вторичка,5,"1,963.00","718,446.60","1,417,386.87","-698,940.27"
11676,186.00,17.00,5.00,ЦАО,Третьяковская,False,премиум,пешком,совмещенный,Кирпичный,Вторичка,5,"1,912.00","586,005.38","1,280,389.54","-694,384.16"
13581,190.00,20.00,6.00,ЦАО,Чистые пруды,False,премиум,пешком,совмещенный,No,Вторичка,5,"1,917.00","421,052.63","1,095,080.69","-674,028.06"
14299,164.00,18.00,6.00,ЗАО,Киевская,False,элитный,на транспорте,совмещенный,Монолитный,Вторичка,5,"2,011.00","701,219.51","1,310,915.60","-609,696.09"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4571,234.00,60.00,7.00,ЦАО,Киевская,False,элитный,на транспорте,раздельный,Монолитный,Новостройка,2,"2,026.00","2,390,384.62","1,485,262.05","905,122.56"
9604,202.00,12.00,6.00,ЦАО,Парк Культуры,False,элитный,пешком,совмещенный,Кирпичный,Вторичка,3,"2,009.00","2,289,287.87","1,197,168.65","1,092,119.23"
10444,120.00,12.00,1.00,ЦАО,Лубянка,False,элитный,пешком,совмещенный,No,Вторичка Апартаменты,3,-1.00,"2,337,411.54","1,063,650.84","1,273,760.70"
13558,302.00,10.00,6.00,ЦАО,Пушкинская,False,элитный,пешком,совмещенный,No,Вторичка,5,"2,006.00","2,423,663.58","1,129,783.45","1,293,880.12"


### Decision Tree ###

In [131]:
dt_reg = DecisionTreeRegressor(random_state = 0, max_depth=15).fit(X_train_transformed, y_train)
MAE_dt = round(mean_absolute_error(y_test, dt_reg.predict(X_test_transformed)), 3)
MAE_dt

36094.482

In [132]:
df_test['preds'] = dt_reg.predict(X_test_transformed)
df_test.head()

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,rooms,year_of_construction,real,preds,diff
16769,7.00,2.00,13.00,ЗАО,Солнцево,False,бизнес,пешком,совмещенный,Блочный,Вторичка Апартаменты,9,"1,978.00","321,428.57","292,307.69","30,149.60"
2532,37.00,10.00,7.00,ЗАО,Раменки,False,бизнес,пешком,совмещенный,Блочный,Вторичка,1,"1,980.00","327,027.03","323,145.86","-42,356.14"
11934,169.00,25.00,9.00,ЦАО,Выставочная,True,элитный,пешком,совмещенный,Монолитный,Новостройка,5,"2,023.00","1,202,130.18","1,015,570.37","166,222.83"
7643,42.00,10.00,1.00,ЦАО,Электрозаводская,False,премиум,пешком,совмещенный,Монолитный,Новостройка,2,"2,024.00","442,837.14","415,627.56","-26,413.48"
17832,21.00,5.00,15.00,ЮАО,ЗИЛ,False,премиум,пешком,совмещенный,Монолитный,Новостройка,9,"2,023.00","677,945.52","474,703.20","190,628.63"


In [133]:
df_test['diff'] = df_test['real']-df_test['preds']
df_test.sort_values(by='diff')

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,rooms,year_of_construction,real,preds,diff
14633,200.00,60.00,5.00,ЦАО,Пушкинская,False,элитный,пешком,совмещенный,Кирпичный,Вторичка,5,"1,908.00","1,095,000.00","2,358,713.19","-1,263,713.19"
9060,150.00,15.00,10.00,ЦАО,Третьяковская,False,элитный,пешком,совмещенный,Монолитный,Вторичка Апартаменты,3,"2,019.00","1,193,333.33","2,256,064.86","-1,062,731.53"
9899,188.00,12.00,12.00,ЦАО,Александровский сад,False,элитный,пешком,совмещенный,Монолитный,Новостройка,3,"2,023.00","1,526,937.70","2,462,114.79","-935,177.09"
11673,239.00,82.00,12.00,ЦАО,Спортивная,False,элитный,пешком,совмещенный,Монолитный,Новостройка Апартаменты,5,"2,024.00","1,614,965.25","2,438,888.89","-823,923.64"
11945,352.00,32.00,16.00,ЦАО,Улица 1905 года,False,элитный,пешком,совмещенный,No,Вторичка Пентхаус,5,"2,016.00","707,386.36","1,526,191.91","-818,805.55"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13844,202.00,18.00,8.00,ЦАО,Спортивная,False,элитный,пешком,совмещенный,Монолитный,Вторичка,5,"2,016.00","1,980,198.02","1,091,957.66","888,240.36"
10459,110.00,4.00,13.00,ЦАО,Киевская,False,элитный,пешком,совмещенный,Монолитный,Новостройка,3,"2,024.00","2,329,340.05","1,391,251.86","938,088.18"
10461,82.00,6.00,13.00,ЦАО,Киевская,False,элитный,пешком,совмещенный,Монолитный,Новостройка,3,"2,024.00","2,344,408.59","1,391,251.86","953,156.72"
9604,202.00,12.00,6.00,ЦАО,Парк Культуры,False,элитный,пешком,совмещенный,Кирпичный,Вторичка,3,"2,009.00","2,289,287.87","1,145,113.04","1,144,174.83"


**Среди базовых моделей лучший результат без настройки гиперпараметров по метрике MAE показывают решающие деревья. Проведем настройку гиперпараметров по кросс-валидации**

In [134]:
from sklearn.model_selection import GridSearchCV

In [135]:
criterion = ['squared_error', 'absolute_error']
splitter = ['best', 'random']
max_depth = [5, 10, 20, 30, 40, 50]

In [136]:
grid = [{'criterion':criterion,
        'splitter':splitter,
        'max_depth':max_depth}]

gs = GridSearchCV(estimator=DecisionTreeRegressor(),
                  param_grid = grid,
                  scoring='neg_mean_absolute_error',
                  cv = 10,
                  n_jobs = -1)

In [137]:
gs = gs.fit(X_train_transformed, y_train)

In [138]:
-1*gs.best_score_

39487.278697835776

In [139]:
gs.best_params_

{'criterion': 'squared_error', 'max_depth': 40, 'splitter': 'random'}

In [140]:
gs.score(X_test_transformed, y_test)

-35255.01214479556

In [141]:
gs.best_estimator_

DecisionTreeRegressor(max_depth=40, splitter='random')

In [142]:
df_test['preds'] = gs.best_estimator_.predict(X_test_transformed)
df_test.head()

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,rooms,year_of_construction,real,preds,diff
16769,7.00,2.00,13.00,ЗАО,Солнцево,False,бизнес,пешком,совмещенный,Блочный,Вторичка Апартаменты,9,"1,978.00","321,428.57","292,307.69","29,120.88"
2532,37.00,10.00,7.00,ЗАО,Раменки,False,бизнес,пешком,совмещенный,Блочный,Вторичка,1,"1,980.00","327,027.03","303,333.33","3,881.17"
11934,169.00,25.00,9.00,ЦАО,Выставочная,True,элитный,пешком,совмещенный,Монолитный,Новостройка,5,"2,023.00","1,202,130.18","1,103,905.33","186,559.81"
7643,42.00,10.00,1.00,ЦАО,Электрозаводская,False,премиум,пешком,совмещенный,Монолитный,Новостройка,2,"2,024.00","442,837.14","415,627.56","27,209.58"
17832,21.00,5.00,15.00,ЮАО,ЗИЛ,False,премиум,пешком,совмещенный,Монолитный,Новостройка,9,"2,023.00","677,945.52","503,532.00","203,242.32"


In [143]:
df_test['diff'] = df_test['real']-df_test['preds']
df_test.sort_values(by='diff')

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,rooms,year_of_construction,real,preds,diff
13013,300.00,18.00,4.00,ЦАО,Театральная,False,элитный,пешком,совмещенный,Монолитный,Вторичка Пентхаус,5,"2,016.00","1,275,000.00","2,841,545.51","-1,566,545.51"
11669,208.00,9.00,12.00,ЦАО,Спортивная,False,элитный,пешком,совмещенный,Монолитный,Новостройка Апартаменты,5,"2,024.00","1,719,338.83","3,137,488.49","-1,418,149.66"
919,115.00,8.00,3.00,ЦАО,Третьяковская,False,элитный,пешком,совмещенный,Монолитный,Новостройка,1,"2,024.00","2,204,017.39","3,000,944.88","-796,927.49"
12750,204.00,18.00,5.00,ЦАО,Полянка,False,элитный,пешком,совмещенный,Кирпичный,Вторичка,5,"2,005.00","735,294.12","1,421,058.59","-685,764.47"
13849,177.00,18.00,5.00,ЦАО,Арбатская,False,элитный,пешком,совмещенный,No,Вторичка,5,"1,914.00","1,124,293.79","1,800,000.00","-675,706.21"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13032,197.00,18.00,6.00,ЦАО,Парк Культуры,False,элитный,пешком,совмещенный,Монолитный,Вторичка,5,"2,010.00","1,664,514.06","716,167.40","948,346.66"
4567,85.00,28.00,5.00,ЦАО,Полянка,False,элитный,пешком,совмещенный,Монолитный,Новостройка,2,"2,024.00","1,900,705.88","898,305.08","1,002,400.80"
13558,302.00,10.00,6.00,ЦАО,Пушкинская,False,элитный,пешком,совмещенный,No,Вторичка,5,"2,006.00","2,423,663.58","968,750.00","1,454,913.58"
10444,120.00,12.00,1.00,ЦАО,Лубянка,False,элитный,пешком,совмещенный,No,Вторичка Апартаменты,3,-1.00,"2,337,411.54","825,000.00","1,512,411.54"


In [144]:
df_test['price_real'] = df_test['total_meters'] * df_test['real']
df_test['price_predicted'] = df_test['total_meters'] * df_test['preds']
df_test['diff'] = df_test['price_real'] - df_test['price_predicted']

In [145]:
df_test

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,rooms,year_of_construction,real,preds,diff,price_real,price_predicted
16769,7.00,2.00,13.00,ЗАО,Солнцево,False,бизнес,пешком,совмещенный,Блочный,Вторичка Апартаменты,9,"1,978.00","321,428.57","292,307.69","203,846.15","2,250,000.00","2,046,153.85"
2532,37.00,10.00,7.00,ЗАО,Раменки,False,бизнес,пешком,совмещенный,Блочный,Вторичка,1,"1,980.00","327,027.03","303,333.33","876,666.67","12,100,000.00","11,223,333.33"
11934,169.00,25.00,9.00,ЦАО,Выставочная,True,элитный,пешком,совмещенный,Монолитный,Новостройка,5,"2,023.00","1,202,130.18","1,103,905.33","16,600,000.00","203,160,000.00","186,560,000.00"
7643,42.00,10.00,1.00,ЦАО,Электрозаводская,False,премиум,пешком,совмещенный,Монолитный,Новостройка,2,"2,024.00","442,837.14","415,627.56","1,142,802.44","18,599,160.00","17,456,357.56"
17832,21.00,5.00,15.00,ЮАО,ЗИЛ,False,премиум,пешком,совмещенный,Монолитный,Новостройка,9,"2,023.00","677,945.52","503,532.00","3,662,684.00","14,236,856.00","10,574,172.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17326,24.00,2.00,19.00,ЗАО,Строгино,False,бизнес,пешком,совмещенный,Монолитный,Новостройка Апартаменты,9,"2,023.00","337,585.00","348,208.57","-254,965.71","8,102,040.00","8,357,005.71"
2939,24.00,10.00,11.00,СВАО,Алексеевская,False,премиум,пешком,совмещенный,Монолитный,Новостройка,1,"2,024.00","528,937.50","421,052.63","2,589,236.84","12,694,500.00","10,105,263.16"
12999,94.00,22.00,20.00,НАО (Новомосковский),Коммунарка,False,эконом,пешком,совмещенный,Панельный,Вторичка,5,"2,021.00","196,808.51","196,808.51",-0.00,"18,500,000.00","18,500,000.00"
6745,58.00,21.00,4.00,СЗАО,Пятницкое шоссе,False,эконом,на транспорте,совмещенный,Монолитный,Новостройка,2,"2,025.00","213,516.19","214,177.47","-38,354.00","12,383,939.00","12,422,293.00"


### CatBoost ###

In [146]:
X_train_ctb = X_train.copy()
X_test_ctb = X_test.copy()

In [147]:
X_train_ctb.dtypes

total_meters            float64
kitchen_meters          float64
dist_to_subway, min     float64
admin_okrug              object
subway                   object
is_skyscraper              bool
class_real               object
way_to_subway            object
wc_type                  object
house_type               object
flat_type                object
rooms                     int64
year_of_construction    float64
dtype: object

In [148]:
X_test_ctb.dtypes

total_meters            float64
kitchen_meters          float64
dist_to_subway, min     float64
admin_okrug              object
subway                   object
is_skyscraper              bool
class_real               object
way_to_subway            object
wc_type                  object
house_type               object
flat_type                object
rooms                     int64
year_of_construction    float64
dtype: object

In [154]:
X_train_ctb.is_skyscraper = X_train_ctb.is_skyscraper.astype(str)
X_test_ctb.is_skyscraper = X_test_ctb.is_skyscraper.astype(str)

In [155]:
X_train_ctb.dtypes == 'O'

total_meters            False
kitchen_meters          False
dist_to_subway, min     False
admin_okrug              True
subway                   True
is_skyscraper            True
class_real               True
way_to_subway            True
wc_type                  True
house_type               True
flat_type                True
rooms                   False
year_of_construction    False
dtype: bool

In [156]:
ctbst = ctb.CatBoostRegressor(cat_features = list(X_train_ctb.columns[X_train_ctb.dtypes == 'O']),
                                 random_state = 42)
ctbst.fit(X_train_ctb, y_train, verbose = False)

In [159]:
ctbst_preds = ctbst.predict(X_test_ctb)

In [161]:
df_test['preds'] = ctbst_preds
df_test.head()

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,rooms,year_of_construction,real,preds,diff,price_real,price_predicted
16769,7.00,2.00,13.00,ЗАО,Солнцево,False,бизнес,пешком,совмещенный,Блочный,Вторичка Апартаменты,9,"1,978.00","321,428.57","335,639.60","203,846.15","2,250,000.00","2,046,153.85"
2532,37.00,10.00,7.00,ЗАО,Раменки,False,бизнес,пешком,совмещенный,Блочный,Вторичка,1,"1,980.00","327,027.03","347,377.10","876,666.67","12,100,000.00","11,223,333.33"
11934,169.00,25.00,9.00,ЦАО,Выставочная,True,элитный,пешком,совмещенный,Монолитный,Новостройка,5,"2,023.00","1,202,130.18","1,105,073.79","16,600,000.00","203,160,000.00","186,560,000.00"
7643,42.00,10.00,1.00,ЦАО,Электрозаводская,False,премиум,пешком,совмещенный,Монолитный,Новостройка,2,"2,024.00","442,837.14","454,542.10","1,142,802.44","18,599,160.00","17,456,357.56"
17832,21.00,5.00,15.00,ЮАО,ЗИЛ,False,премиум,пешком,совмещенный,Монолитный,Новостройка,9,"2,023.00","677,945.52","485,591.19","3,662,684.00","14,236,856.00","10,574,172.00"


In [162]:
MAE_ctbst = round(mean_absolute_error(y_test, ctbst_preds), 3)
MAE_ctbst

38221.444

In [163]:
ctbst2 = ctb.CatBoostRegressor(iterations = 1000, 
                             depth = 15, 
                             learning_rate = 0.04746000096201897,
                             random_strength = 1,
                             border_count = 254,
                             l2_leaf_reg = 3, 
                             grow_policy = 'SymmetricTree',
                             cat_features = list(X_train_ctb.columns[X_train_ctb.dtypes == 'O']),
                             random_state = 42)
ctbst2.fit(X_train_ctb, y_train, verbose = False)

In [164]:
ctbst2_preds = ctbst2.predict(X_test_ctb)
MAE_ctbst2 = round(mean_absolute_error(y_test, ctbst2_preds), 3)
MAE_ctbst2

31705.333

In [166]:
df_test['preds'] = ctbst2_preds
df_test.head()

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,rooms,year_of_construction,real,preds,diff,price_real,price_predicted
16769,7.00,2.00,13.00,ЗАО,Солнцево,False,бизнес,пешком,совмещенный,Блочный,Вторичка Апартаменты,9,"1,978.00","321,428.57","339,364.83","203,846.15","2,250,000.00","2,046,153.85"
2532,37.00,10.00,7.00,ЗАО,Раменки,False,бизнес,пешком,совмещенный,Блочный,Вторичка,1,"1,980.00","327,027.03","355,752.71","876,666.67","12,100,000.00","11,223,333.33"
11934,169.00,25.00,9.00,ЦАО,Выставочная,True,элитный,пешком,совмещенный,Монолитный,Новостройка,5,"2,023.00","1,202,130.18","1,102,728.17","16,600,000.00","203,160,000.00","186,560,000.00"
7643,42.00,10.00,1.00,ЦАО,Электрозаводская,False,премиум,пешком,совмещенный,Монолитный,Новостройка,2,"2,024.00","442,837.14","432,314.43","1,142,802.44","18,599,160.00","17,456,357.56"
17832,21.00,5.00,15.00,ЮАО,ЗИЛ,False,премиум,пешком,совмещенный,Монолитный,Новостройка,9,"2,023.00","677,945.52","549,156.31","3,662,684.00","14,236,856.00","10,574,172.00"


### RandomForest Regressor ###

In [167]:
rf = RandomForestRegressor(criterion='squared_error', n_estimators = 300)

In [168]:
rf.fit(X_train_transformed, y_train)

RandomForestRegressor(n_estimators=300)

In [169]:
rf_preds = rf.predict(X_test_transformed)
MAE_rf = round(mean_absolute_error(y_test, rf_preds), 3)
MAE_rf

29476.033

In [171]:
import pickle
with open("rf_reg.pkl", "wb") as f:
    pickle.dump(rf, f)

In [172]:
with open("rf_reg.pkl", "rb") as f:
    model = pickle.load(f)

In [181]:
sample_test = {
    'total_meters':[58.2],
    'kitchen_meters':[21.0],
    'dist_to_subway, min':[15.0],
    'admin_okrug':['СВАО'],
    'subway':['Ботанический сад'],
    'is_skyscraper': ['False'],
    'class_real':['комфорт'],
    'way_to_subway':['пешком'],
    'wc_type':['совмещенный'],
    'house_type':['Монолитный'],
    'flat_type':['Вторичка'],
    'rooms': [2],
    'year_of_construction':[2021]
    
}

In [182]:
sample_df = pd.DataFrame.from_dict(sample_test)
sample_df

,total_meters,kitchen_meters,"dist_to_subway, min",admin_okrug,subway,is_skyscraper,class_real,way_to_subway,wc_type,house_type,flat_type,rooms,year_of_construction
0,58.20,21.00,15.00,СВАО,Ботанический сад,False,комфорт,пешком,совмещенный,Монолитный,Вторичка,2,2021


In [183]:
sample_df_transformed = col_transformer.transform(sample_df)

In [187]:
model.predict(sample_df_transformed)[0] * 58.2

15176652.90611582

In [203]:
# ### Гиперпараметры леса
# choices = pd.DataFrame(index = range(0, 1000, 1), columns = ['n_estimators',
#                                                             'max_depth',
#                                                             'min_samples_split',
#                                                             'min_samples_leaf',
#                                                             'max_features',
#                                                             'max_leaf_nodes',
#                                                             'bootstrap',
#                                                             'max_samples',
#                                                             'threshold'])
# # 'special' for None
# choices['n_estimators'] = np.random.RandomState(42).choice([100, 300, 500, 700, 1000]*200, 1000, replace = False)
# choices['max_depth'] = np.random.RandomState(43).choice([2, 4, 6, 8, 10]*200, 1000, replace = False)
# choices['min_samples_split'] = np.random.RandomState(44).choice([2, 5, 10, 20, 40]*200, 1000, replace = False)
# choices['min_samples_leaf'] = np.random.RandomState(45).choice([1, 2, 5, 10, 20]*200, 1000, replace = False)
# choices['max_features'] = np.random.RandomState(46).choice([0.7, 'log2', 'sqrt', 'auto']*250, 1000, replace = False)
# choices['max_leaf_nodes'] = np.random.RandomState(47).choice([5, 10, 20, 'special']*250, 1000, replace = False)
# choices['bootstrap'] = np.random.RandomState(48).choice([True]*800 + [False]*200, 1000, replace = False)
# choices['max_samples'] = np.random.RandomState(49).choice([0.5, 0.7, 0.9, 'special']*250, 1000, replace = False)
# choices['threshold'] = np.random.RandomState(50).choice([0.05, 0.01, 0.005, 0.001]*250, 1000, replace = False)

# choices.loc[~choices.max_features.str.contains('log2|sqrt|auto'), 'max_features'] = \
# choices.loc[~choices.max_features.str.contains('log2|sqrt|auto'), 'max_features'].astype(np.float64)
# choices.loc[choices.max_leaf_nodes != 'special', 'max_leaf_nodes'] = \
# choices.loc[choices.max_leaf_nodes != 'special', 'max_leaf_nodes'].astype(np.int64)
# choices.loc[choices.max_samples != 'special', 'max_samples'] = \
# choices.loc[choices.max_samples != 'special', 'max_samples'].astype(np.float64)

# ### Гиперпараметры бустинга
# choices_bstr = pd.DataFrame(index = range(0, 500, 1), columns = ['iterations', 
#                                                                  'depth',  
#                                                                  'learning_rate', 
#                                                                  'random_strength',  
#                                                                  'bagging_temperature', 
#                                                                  'border_count',  
#                                                                  'l2_leaf_reg',  
#                                                                  'grow_policy',  
#                                                                  'threshold'])
# choices_bstr['iterations'] = np.random.RandomState(42).choice([250, 500, 1000, 1500]*125, 500, replace = False)
# choices_bstr['depth'] = np.random.RandomState(43).choice([2, 4, 6, 8]*125, 500, replace = False)
# choices_bstr['learning_rate'] = np.random.RandomState(44).choice([0.001, 0.01, 0.1, 0.2, 0.3]*100, 500, replace = False)
# choices_bstr['random_strength'] = np.random.RandomState(45).choice([0, 0.3, 0.6, 1]*125, 500, replace = False)
# choices_bstr['bagging_temperature'] = np.random.RandomState(46).choice([0, 0.7, 0.9, 1]*125, 500, replace = False)
# choices_bstr['border_count'] = np.random.RandomState(47).choice([60, 128, 200, 254]*125, 500, replace = False)
# choices_bstr['l2_leaf_reg'] = np.random.RandomState(48).choice([1, 3, 5, 10]*125, 500, replace = False)
# choices_bstr['grow_policy'] = np.random.RandomState(49).choice(['SymmetricTree', 'Depthwise']*250, 500, replace = False)
# choices_bstr['threshold'] = np.random.RandomState(50).choice([0.01, 0.005, 0.001, 0.0005]*125, 500, replace = False)

**Можно сказать, что наибольшей важностью для правильности предсказания уровня арендной платы являются:**
- площадь квартиры
- административный район Москвы (расположение в ЦАО существеннее всего влияет на арендную плату)
- этаж и количество этажей в доме
- наличие посудомоечной машины